In [1]:
from el_toolkit.document import Document
from el_toolkit.lkb.lexical_knowledge_base import Lexical_Knowledge_Base
from transformers import BertTokenizer


lkb = Lexical_Knowledge_Base.read_json("data/knowledge_datasets/mesh_subset.json")

biomed_tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path='monologg/biobert_v1.1_pubmed',
                                            do_lower_case=False, cache_dir=None)
                                            
doc = Document.read_json("/Users/coltonflowers/Work/TREND_repos/BioMedical-EL/tests/test_data/test_docs/without_overlaps/14_mentions.json")
doc.jupyter_display(lkb)
for doc in doc.segment(biomed_tokenizer,8):
  doc.jupyter_display(lkb)
  print("-----------------------------------")

Mentions Omitted: 0


-----------------------------------


-----------------------------------


In [2]:
from el_toolkit.entity_linkers.dual_embedder.featurizer import BertDualEmbedderTrainFeaturizer
from el_toolkit.entity_linkers.dual_embedder.concept_embedder import BertConceptEmbedder
from el_toolkit.entity_linkers.dual_embedder.document_embedder import DocumentEmbedder
from el_toolkit.entity_linkers.dual_embedder.entity_linker import DualEmbedderEntityLinker
from el_toolkit.entity_linkers.dual_embedder.model import BertMentionDetectorModel
from el_toolkit.entity_linkers.dual_embedder.model import BertDualEmbedderModel
from transformers import BertModel
import copy

bert_mention = BertModel.from_pretrained("monologg/biobert_v1.1_pubmed")
bert_candidate = copy.deepcopy(bert_mention)
concept_embedder = BertConceptEmbedder(lkb,bert_model=bert_mention,tokenizer=biomed_tokenizer,max_seq_len=256)
span_detector = BertMentionDetectorModel(bert_mention)
document_embedder = DocumentEmbedder(span_detector=span_detector,tokenizer=biomed_tokenizer,max_seq_len=256)
dual_embedder_model = BertDualEmbedderModel(bert_mention,bert_candidate)
dual_embedder = DualEmbedderEntityLinker(concept_embedder,document_embedder,dual_embedder_model)

featurizer = BertDualEmbedderTrainFeaturizer(dual_embedder,lkb=lkb,num_hard_negatives=0,num_random_negatives=8,num_max_mentions=8)

Some weights of the model checkpoint at monologg/biobert_v1.1_pubmed were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
docs = doc.segment(biomed_tokenizer,8)
dataset = [featurizer.featurize(docs)]

Mentions Omitted: 0


In [5]:
features = featurizer.featurize_doc(docs[0])
features

TrainingInputFeatures(doc_token_ids=[101, 1170, 1210, 4776, 3469, 1104, 23280, 3276, 15789, 1566, 1170, 1210, 4776, 3469, 1104, 23280, 3276, 15789, 1566, 119, 18732, 15517, 2162, 13329, 24805, 131, 1284, 17581, 1115, 4420, 1114, 184, 13894, 4184, 14824, 4863, 1170, 1210, 4776, 3469, 1104, 23280, 3276, 15789, 1566, 119, 18732, 15517, 2162, 13329, 24805, 131, 1284, 17581, 1115, 4420, 1114, 184, 13894, 4184, 14824, 4863, 1169, 2592, 2489, 1170, 1210, 4776, 3469, 1104, 23280, 3276, 15789, 1566, 119, 18732, 15517, 2162, 13329, 24805, 131, 1284, 17581, 1115, 4420, 1114, 184, 13894, 4184, 14824, 4863, 1169, 2592, 2489, 117, 1105, 16516, 20080, 15342, 21250, 2193, 1170, 1210, 4776, 3469, 1104, 23280, 3276, 15789, 1566, 119, 18732, 15517, 2162, 13329, 24805, 131, 1284, 17581, 1115, 4420, 1114, 184, 13894, 4184, 14824, 4863, 1169, 2592, 2489, 117, 1105, 16516, 20080, 15342, 21250, 2193, 118, 2272, 2489, 1170, 1210, 4776, 3469, 1104, 23280, 3276, 15789, 1566, 119, 18732, 15517, 2162, 13329, 24805

In [2]:
from el_toolkit.entity_linkers.dual_embedder.model import BertMentionDetectorModel,BertDualEmbedderModel
detector_model = BertMentionDetectorModel.from_pretrained_bert_filepath("monologg/biobert_v1.1_pubmed")
dual_embedder = BertDualEmbedderModel.from_pretrained_bert_filepath("monologg/biobert_v1.1_pubmed")

Some weights of the model checkpoint at monologg/biobert_v1.1_pubmed were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at monologg/biobert_v1.1_pubmed were not used when initializing BertModel: ['cls.seq_relationship.weight', 